## install

In [ ]:
!git lfs install
!git clone https://github.com/index-tts/index-tts.git
!cd /content/index-tts && git lfs pull
!pip install -U uv
!cd /content/index-tts && uv sync --all-extras

!uv tool install "huggingface-hub[cli,hf_xet]"
!hf download IndexTeam/IndexTTS-2 --local-dir=/content/index-tts/checkpoints

In [ ]:
!cd /content/index-tts && uv python install 3.11 && uv tool install spleeter
!wget -O /content/index-tts/2stems.tar.gz https://github.com/deezer/spleeter/releases/download/v1.4.0/2stems.tar.gz

## API Cloudflare

In [4]:
import os
from google.colab import userdata

API = userdata.get('API')
TOKEN = userdata.get('TOKEN')

os.environ['CLOUDFLARE_API'] = API
os.environ['CLOUDFLARE_TOKEN'] = TOKEN

## main

In [ ]:
import os
with open('index-tts/api.py', 'w') as f:
  f.write(
      '''
import os
import re
import ffmpy
import shutil
import tarfile
import requests
import subprocess
import gradio as gr
from base64 import b64encode
from indextts.infer_v2 import IndexTTS2
if not os.path.exists('pretrained_models'):
    with tarfile.open('2stems.tar.gz', 'r:gz') as tar_ref:
        tar_ref.extractall('./pretrained_models/2stems')
API = os.environ.get('CLOUDFLARE_API')
TOKEN = os.environ.get('CLOUDFLARE_TOKEN')
API_BASE_URL = f"https://api.cloudflare.com/client/v4/accounts/{API}/ai/run/"
headers = {"Authorization": f"Bearer {TOKEN}"}
tts = IndexTTS2(cfg_path="/content/index-tts/checkpoints/config.yaml", model_dir="checkpoints", use_fp16=False, use_cuda_kernel=False, use_deepspeed=False)
file_name = 'audio'
folder = 'output_directory'
main_video = f'./{folder}/video.mp4'
main_audio = f'./{folder}/{file_name}.wav'
vocals = f'./{folder}/{file_name}/vocals.wav'
vocals_monorail = f'./{folder}/{file_name}/vocals_monorail.wav'
voice_imitation = f'./{folder}/{file_name}/voice_imitation.wav'
splitting_path = f'./{folder}/{file_name}/splitting/audio_splitting'
translated_path = f'./{folder}/{file_name}/translated/audio_translated'
mute = f'./{folder}/{file_name}/mute/'
accompaniment = f'./{folder}/{file_name}/accompaniment.wav'
output_audio = f'./{folder}/output.wav'
output_video = f'./{folder}/output.mp4'
def create_directory():
  os.mkdir('output_directory')
  os.mkdir(f'output_directory/{file_name}')
  os.mkdir(f'output_directory/{file_name}/splitting')
  os.mkdir(f'output_directory/{file_name}/translated')
  os.mkdir(f'output_directory/{file_name}/mute')
def ffmpeg_d(input, output):
  ff = ffmpy.FFmpeg(
  inputs = input,
  outputs = output
  )
  ff.run()
def gain_time(audio):
  command = ['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', audio]
  result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  return result.stdout.decode().strip()

def left_justified(audio):
  try:
    command = ['ffmpeg', '-i', audio, '-af', 'silencedetect=n=-38dB:d=0.01', '-f', 'null', '-']
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    start_justified = re.search(r'silence_duration: (\d.\d+)', result.stdout.decode(), re.M|re.S).group(1)
  except AttributeError:
    raise gr.Error('left_justified No start sound detected!')
  return start_justified
def right_justified(audio):
  try:
    command = ['ffmpeg', '-i', audio, '-af', 'areverse,silencedetect=n=-38dB:d=0.01,areverse', '-f', 'null', '-']
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    end_justified = re.search(r'silence_duration: (\d.\d+)', result.stdout.decode(), re.M|re.S).group(1)
  except AttributeError:
    raise gr.Error('right_justified No start sound detected!')
  return end_justified
def time_verify(vocals_audio, target_audio):
  audios = [vocals_audio, target_audio]
  justified = []
  time_lists = []
  for audio in audios:
    justified.append(left_justified(audio))
    time_lists.append(gain_time(audio))
  j_time = float(justified[0]) - float(justified[1])
  if float(time_lists[0]) > float(time_lists[1]):
      r_time = float(min(time_lists)) / (float(max(time_lists)) - j_time)
  else:
      r_time = float(max(time_lists)) / float(min(time_lists))
  return round(j_time, 6), round(r_time, 6)
def run(model, inputs):
  response = requests.post(f"{API_BASE_URL}{model}", headers=headers, json=inputs)
  if response.status_code != 200:
      raise gr.Error(f"API call failed with status code {response.status_code}")
  return response.json()
def audio_texts(audio):
  time_period_list = list()
  sound = open(audio, "rb").read()
  sound_base64 = b64encode(sound).decode()
  inputs = {
      "audio": sound_base64,
      "task": "transcribe"
  }
  output = run("@cf/openai/whisper-large-v3-turbo", inputs)
  language = output['result']['transcription_info']['language']
  texts = output['result']['text']
  vtt = output['result']['vtt']
  print(language)
  print(texts)
  print(vtt)
  for segment in output['result']['segments']:
      time_period = dict({'start': segment['start'], 'end': segment['end'], 'text': segment['text']})
      time_period_list.append(time_period)
  for i in range(len(time_period_list)):
    time_period_list[i]['end'] = float(gain_time(vocals_monorail)) if i == len(time_period_list) - 1 else time_period_list[i]['end']
  print('✅ time_period_list:', time_period_list)
  return language, texts, vtt, time_period_list
def translate_texts(texts, language):
  ts_texts = list()
  prompt = """
  You are a professional {{to}} native translator who needs to fluently translate text into {{to}}.

  ## Translation Rules
  1. Output only the translated content, without explanations or additional content (such as "Here's the translation:" or "Translation as follows:")
  2. The returned translation must maintain exactly the same number of paragraphs and format as the original text
  3. If the text contains HTML tags, consider where the tags should be placed in the translation while maintaining fluency
  4. For content that should not be translated (such as proper nouns, code, etc.), keep the original text.
  5. If input contains %%, use %% in your output, if input has no %%, don't use %% in your output{{title_prompt}}{{summary_prompt}}{{terms_prompt}}

  ## OUTPUT FORMAT:
  - **Single paragraph input** → Output translation directly (no separators, no extra text)
  - **Multi-paragraph input** → Use %% as paragraph separator between translations

  ## Examples
  ### Multi-paragraph Input:
  Paragraph A
  %%
  Paragraph B
  %%
  Paragraph C
  %%
  Paragraph D

  ### Multi-paragraph Output:
  Translation A
  %%
  Translation B
  %%
  Translation C
  %%
  Translation D

  ### Single paragraph Input:
  Single paragraph content

  ### Single paragraph Output:
  Direct translation without separators
  """
  for text in range(len(texts)):
    response = requests.post(
    f"https://api.cloudflare.com/client/v4/accounts/{API}/ai/v1/responses",
      headers={"Authorization": f"Bearer {TOKEN}"},
      json={
        "model": "@cf/openai/gpt-oss-120b",
        "input": f"Translate to {language} ({texts[text]['text']})"
      }
    )
    result = response.json()
    ts_texts.append(result['output'][1]['content'][0]['text'])
  print('✅ ts_texts:', ts_texts)
  return ts_texts
def audio_splitting(time_period_list):
  for i in range(len(time_period_list)):
    ffmpeg_d(
        {vocals_monorail: None},
        {
        f'{splitting_path}_{i}.wav': ['-i', vocals_monorail, '-ss', f"{time_period_list[i]['start']}", '-to', f"{time_period_list[i]['end']}", '-codec', 'copy', '-f', 'wav']
        }
  )
def audio_translate(time_period_list, ts_texts):
  try:
    for a in range(len(ts_texts)):
      tts.infer(spk_audio_prompt=f'{splitting_path}_{a}.wav', text=ts_texts[a], output_path=f'{translated_path}_{a}.wav', verbose=True)
  except Exception as e:
    raise gr.Error(f'✅ time_period_list: {time_period_list}\\n ✅ ts_texts: {ts_texts}\\n ⚠️ {splitting_path}_{a}.wav and {time_period_list[a]} and {ts_texts[a]}\\n ❌ Error: {e}')
def video_inputs(video, LANGUAGE):
  if not os.path.exists('output_directory'):
      create_directory()
  else:
    shutil.rmtree('output_directory')
    create_directory()
  ffmpeg_d(
      {video: None},
      {
      main_video: ['-y', '-map', '0:0', '-c:a', 'copy', '-f', 'mp4'],
      main_audio: ['-y', '-map', '0:a', '-vn', '-acodec', 'pcm_s16le', '-ar', '16000', '-ac', '1', '-f', 'wav']
      }
  )
  subprocess.run(['uv', 'tool', 'run', 'spleeter', 'separate', '-o', folder, '-p', 'spleeter:2stems-16kHz', main_audio])
  ffmpeg_d(
      {vocals: None},
      {
      vocals_monorail: ['-y', '-vn', '-acodec', 'pcm_s16le', '-ar', '16000', '-ac', '1', '-f', 'wav']
      }
  )
  language, texts, vtt, time_period_list = audio_texts(vocals_monorail)
  audio_splitting(time_period_list)
  ts_texts = translate_texts(time_period_list, LANGUAGE)
  audio_translate(time_period_list, ts_texts)
  message = list()
  for j in range(len(time_period_list)):
    ffmpeg_d(
      {f'{translated_path}_{j}.wav': None},
      {
      f'{mute}left_audio_{j}.wav': ['-y', '-af', f"atrim=start={left_justified(f'{translated_path}_{j}.wav')}"]
      }
    )
    ffmpeg_d(
      {f'{mute}left_audio_{j}.wav': None},
      {
      f'{mute}right_audio_{j}.wav': ['-y', '-af', f"areverse,atrim=start={right_justified(f'{translated_path}_{j}.wav')},areverse"]
      }
    )
    left_justified_time = float(left_justified(f'{splitting_path}_{j}.wav'))
    if left_justified_time > 0:
      ffmpeg_d(
        {f'{mute}right_audio_{j}.wav': None},
        {
        f'{mute}output_left_audio_{j}.wav': ['-y', '-af', f'areverse,apad=pad_dur={left_justified_time}s,areverse']
        }
      )
      message.append(f"✅ output_left_audio_{j}.wav: {time_period_list[j]['start']}-{time_period_list[j]['end']} : {left_justified_time}")
    else:
      ffmpeg_d(
        {f'{mute}right_audio_{j}.wav': None},
        {
        f'{mute}output_left_audio_{j}.wav': ['-y',  '-filter:a', f'atrim=start={abs(left_justified_time)}']
        }
      )
      message.append(f"⚠️ output_left_audio_{j}.wav: {time_period_list[j]['start']}-{time_period_list[j]['end']} : {left_justified_time}")
    r_time = time_verify(f'{splitting_path}_{j}.wav', f'{mute}output_left_audio_{j}.wav')
    ffmpeg_d(
      {f'{mute}output_left_audio_{j}.wav': None},
      {
      f'{mute}output_rate_audio_{j}.wav': ['-y', '-filter:a', f'atempo={r_time[1]}']
      }
    )
    if j == 0:
      gap_time = float(time_period_list[j+1]['start']) - float(gain_time(f'{mute}output_rate_audio_{j}.wav'))
    if j > 0 and j == len(time_period_list) - 2:
      gap_time = float(time_period_list[j+1]['start']) - (float(gain_time(f'{mute}output_gap_audio{j-1}.wav'))+float(gain_time(f'{mute}output_rate_audio_{j}.wav')))
    elif j > 0 and j == len(time_period_list) - 1:
      gap_time = 0
    if gap_time < 0:
      raise gr.Error(f'⚠️ gap_time is negative: {gap_time} at segment {j}. Please check the time periods and audio lengths.')
    if gap_time > 0 and j == 0:
      ffmpeg_d(
        {f'{mute}output_rate_audio_{j}.wav': None},
        {
        f'{mute}output_gap_audio{j}.wav': ['-y', '-af', f'apad=pad_dur={gap_time}s']
        }
      )
      message.append(f"✅ output_gap_audio{j}.wav: {time_period_list[j]['start']}-{time_period_list[j]['end']} : {gap_time}")
    elif gap_time > 0 and j == len(time_period_list) - 2:
      ffmpeg_d(
        {
        f'{mute}output_gap_audio{j-1}.wav': None,
        f'{mute}output_rate_audio_{j}.wav': None
        },
        {
        f'{mute}output_gap_audio{j}_temp.wav': ['-y', '-filter_complex', '[0:a][1:a]concat=n=2:v=0:a=1[outa]', '-map', '[outa]']
        }
      )
      ffmpeg_d(
        {f'{mute}output_gap_audio{j}_temp.wav': None},
        {
        f'{mute}output_gap_audio{j}.wav': ['-y', '-af', f'apad=pad_dur={gap_time}s']
        }
      )
      message.append(f"✅ output_gap_audio{j}.wav: {time_period_list[j]['start']}-{time_period_list[j]['end']} : {gap_time}")
    else:
      ffmpeg_d(
        {
        f'{mute}output_gap_audio{j-1}.wav': None,
        f'{mute}output_rate_audio_{j}.wav': None
        },
        {
        voice_imitation: ['-y', '-filter_complex', '[0:a][1:a]concat=n=2:v=0:a=1[outa]', '-map', '[outa]']
        }
      )
      message.append(f'✅ output.wav: {time_period_list[j]["start"]}-{time_period_list[j]["end"]}')
  ffmpeg_d(
      {voice_imitation: None, accompaniment: None},
      {
      output_audio: ['-y', '-filter_complex', 'amix=inputs=2']
      }
  )
  ffmpeg_d(
      {output_audio: None, main_video: None},
      {
      output_video: ['-y', '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental']
      }
  )
  print('\\n'.join(message))
  return output_video, vocals_monorail, voice_imitation, accompaniment, ts_texts, time_period_list, vtt
with gr.Blocks() as demo:
  LANGUAGE = gr.Text(value='Chinese (Simplified)', label='Language')
  gr.Interface(
    fn=video_inputs,
    inputs=[
        gr.Video(height=320, interactive=True, label='Input_video'),
        LANGUAGE
        ],
    outputs=[
        gr.Video(height=320, label='Output_video'),
        gr.Audio(label='Vocals_monorail'),
        gr.Audio(label='Voice_imitation'),
        gr.Audio(label='Accompaniment'),
        gr.Text(label='Translator'),
        gr.Text(label='Time_period_list'),
        gr.Text(label='VTT'),
    ],
    title="Short-Video-To-Video API",
    description="🤗 Use the API provided by [Cloudflare](https://developers.cloudflare.com/workers-ai/models/?authors=openai) to call Whisper, and then convert the audio locally through the index-tts function."
  )

if __name__ == "__main__":
  demo.launch(share=True)
      '''
  )

## script

In [ ]:
import os
with open('index-tts/script.py', 'w') as f:
  f.write(
      '''
import re
import ffmpy
import subprocess
from indextts.infer_v2 import IndexTTS2
file_name = 'audio'
folder = 'output_directory'
main_video = f'./{folder}/video.mp4'
vocals = f'./{folder}/{file_name}/vocals.wav'
voice_imitation = f'./{folder}/{file_name}/voice_imitation.wav'
splitting_path = f'./{folder}/{file_name}/splitting/audio_splitting'
translated_path = f'./{folder}/{file_name}/translated/audio_translated'
mute = f'./{folder}/{file_name}/mute/'
accompaniment = f'./{folder}/{file_name}/accompaniment.wav'
output_audio = f'./{folder}/output.wav'
output_video = f'./{folder}/output.mp4'

# Modify parameters before running
time_period_list = [{'start': 0, 'end': 4.06, 'text': 'xxx'}, {'start': 4.46, 'end': 9.28, 'text': 'xxx'}, {'start': 9.54, 'end': 15.325188, 'text': 'xxx'}]
ts_texts = ['xxx', 'xxx', 'xxx']
num = 1

def gain_time(audio):
  command = ['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', audio]
  result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
  return result.stdout.decode().strip()
def left_justified(audio):
  try:
    command = ['ffmpeg', '-i', audio, '-af', 'silencedetect=n=-38dB:d=0.01', '-f', 'null', '-']
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    start_justified = re.search(r'silence_duration: (\d.\d+)', result.stdout.decode(), re.M|re.S).group(1)
  except AttributeError:
    raise Exception('left_justified No start sound detected!')
  return start_justified
def right_justified(audio):
  try:
    command = ['ffmpeg', '-i', audio, '-af', 'areverse,silencedetect=n=-38dB:d=0.01,areverse', '-f', 'null', '-']
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    end_justified = re.search(r'silence_duration: (\d.\d+)', result.stdout.decode(), re.M|re.S).group(1)
  except AttributeError:
    raise Exception('right_justified No start sound detected!')
  return end_justified
def time_verify(vocals_audio, target_audio):
  audios = [vocals_audio, target_audio]
  justified = []
  time_lists = []
  for audio in audios:
    justified.append(left_justified(audio))
    time_lists.append(gain_time(audio))
  j_time = float(justified[0]) - float(justified[1])
  if float(time_lists[0]) > float(time_lists[1]):
      r_time = float(min(time_lists)) / (float(max(time_lists)) - j_time)
  else:
      r_time = float(max(time_lists)) / float(min(time_lists))
  return round(j_time, 6), round(r_time, 6)
def audio_translate(ts_texts):
  tts = IndexTTS2(cfg_path="/content/index-tts/checkpoints/config.yaml", model_dir="checkpoints", use_fp16=False, use_cuda_kernel=False, use_deepspeed=False)
  for a in range(num, len(ts_texts)):
    tts.infer(spk_audio_prompt=f'{splitting_path}_{a}.wav', text=ts_texts[a], output_path=f'{translated_path}_{a}.wav', verbose=True)
def ffmpeg_d(input, output):
  ff = ffmpy.FFmpeg(
  inputs = input,
  outputs = output
  )
  ff.run()
audio_translate(ts_texts)
message = list()
for j in range(len(time_period_list)):
  ffmpeg_d(
    {f'{translated_path}_{j}.wav': None},
    {
    f'{mute}left_audio_{j}.wav': ['-y', '-af', f"atrim=start={left_justified(f'{translated_path}_{j}.wav')}"]
    }
  )
  ffmpeg_d(
    {f'{mute}left_audio_{j}.wav': None},
    {
    f'{mute}right_audio_{j}.wav': ['-y', '-af', f"areverse,atrim=start={right_justified(f'{translated_path}_{j}.wav')},areverse"]
    }
  )
  left_justified_time = float(left_justified(f'{splitting_path}_{j}.wav'))
  if left_justified_time > 0:
    ffmpeg_d(
      {f'{mute}right_audio_{j}.wav': None},
      {
      f'{mute}output_left_audio_{j}.wav': ['-y', '-af', f'areverse,apad=pad_dur={left_justified_time}s,areverse']
      }
    )
    message.append(f"✅ output_left_audio_{j}.wav: {time_period_list[j]['start']}-{time_period_list[j]['end']} : {left_justified_time}")
  else:
    ffmpeg_d(
      {f'{mute}right_audio_{j}.wav': None},
      {
      f'{mute}output_left_audio_{j}.wav': ['-y',  '-filter:a', f'atrim=start={abs(left_justified_time)}']
      }
    )
    message.append(f"⚠️ output_left_audio_{j}.wav: {time_period_list[j]['start']}-{time_period_list[j]['end']} : {left_justified_time}")
  r_time = time_verify(f'{splitting_path}_{j}.wav', f'{mute}output_left_audio_{j}.wav')
  ffmpeg_d(
    {f'{mute}output_left_audio_{j}.wav': None},
    {
    f'{mute}output_rate_audio_{j}.wav': ['-y', '-filter:a', f'atempo={r_time[1]}']
    }
  )
  if j == 0:
    gap_time = float(time_period_list[j+1]['start']) - float(gain_time(f'{mute}output_rate_audio_{j}.wav'))
  if j > 0 and j == len(time_period_list) - 2:
    gap_time = float(time_period_list[j+1]['start']) - (float(gain_time(f'{mute}output_gap_audio{j-1}.wav'))+float(gain_time(f'{mute}output_rate_audio_{j}.wav')))
  elif j > 0 and j == len(time_period_list) - 1:
    gap_time = 0
  if gap_time < 0:
    raise Exception(f'⚠️ gap_time is negative: {gap_time} at segment {j}. Please check the time periods and audio lengths.')
  if gap_time > 0 and j == 0:
    ffmpeg_d(
      {f'{mute}output_rate_audio_{j}.wav': None},
      {
      f'{mute}output_gap_audio{j}.wav': ['-y', '-af', f'apad=pad_dur={gap_time}s']
      }
    )
    message.append(f"✅ output_gap_audio{j}.wav: {time_period_list[j]['start']}-{time_period_list[j]['end']} : {gap_time}")
  elif gap_time > 0 and j == len(time_period_list) - 2:
    ffmpeg_d(
      {
      f'{mute}output_gap_audio{j-1}.wav': None,
      f'{mute}output_rate_audio_{j}.wav': None
      },
      {
      f'{mute}output_gap_audio{j}_temp.wav': ['-y', '-filter_complex', '[0:a][1:a]concat=n=2:v=0:a=1[outa]', '-map', '[outa]']
      }
    )
    ffmpeg_d(
      {f'{mute}output_gap_audio{j}_temp.wav': None},
      {
      f'{mute}output_gap_audio{j}.wav': ['-y', '-af', f'apad=pad_dur={gap_time}s']
      }
    )
    message.append(f"✅ output_gap_audio{j}.wav: {time_period_list[j]['start']}-{time_period_list[j]['end']} : {gap_time}")
  else:
    ffmpeg_d(
      {
      f'{mute}output_gap_audio{j-1}.wav': None,
      f'{mute}output_rate_audio_{j}.wav': None
      },
      {
      voice_imitation: ['-y', '-filter_complex', '[0:a][1:a]concat=n=2:v=0:a=1[outa]', '-map', '[outa]']
      }
    )
    message.append(f'✅ output.wav: {time_period_list[j]["start"]}-{time_period_list[j]["end"]}')
ffmpeg_d(
    {voice_imitation: None, accompaniment: None},
    {
    output_audio: ['-y', '-filter_complex', 'amix=inputs=2']
    }
)
ffmpeg_d(
    {output_audio: None, main_video: None},
    {
    output_video: ['-y', '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental']
    }
)
print('\\n'.join(message))
      '''
  )

## CPU ROM 16G

In [ ]:
!cd /content/index-tts && uv run api.py --model_dir /content/index-tts/checkpoints

In [ ]:
!cd /content/index-tts && uv run script.py --model_dir /content/index-tts/checkpoints

## GPU

In [ ]:
!cd /content/index-tts && uv run api.py --model_dir /content/index-tts/checkpoints --fp16 --deepspeed --cuda_kernel

In [ ]:
!cd /content/index-tts && uv run script.py --model_dir /content/index-tts/checkpoints --fp16 --deepspeed --cuda_kernel